# Reference
Adapted for Kerwa Lake coordinates and geometry based on the code by Pushpendra Kushwaha

https://colab.research.google.com/drive/1A_XxXivyGY4y-ize5adNReYjMEgJgy8Z#scrollTo=0ezYwq74Z9di



In [3]:
 !pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install geehydro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import ee, datetime
import pandas as pd
import numpy as np
import folium
import geehydro
import datetime as dt
import IPython.display


In [6]:
ee.Authenticate()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ufycG8KW1H6HmAP6OBomCJ_EIJ5qljKmzgxlejHgFSg&tc=LpbQSQCKDAEr3fsfatOwyj76amdkwJ6sGOxa1BiPo70&cc=PVjVs_mLKTq9hqo-zzE-sEJQm3AUPXBjkTcmpI6u8_Y

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh4eW-KtLfKczItkUwLyQqDZHHh5s624Ewm8Uj_AVjwZH7tkfiLUUyQ

Successfully saved authorization token.


# Selecting the area of Interest

In [7]:
ee.Initialize()

In [8]:
kerwa = folium.Map(location=[23.167912,77.372411], zoom_start=10)


In [10]:
kerwa

# Selecting the satellite and AOI

In [11]:
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate("2015-01-01","2023-01-01")
kerwa_AOI = ee.FeatureCollection('users/ferreirawg/kerwa_lake_bhopal')

sentinel_AOI = sentinel.filterBounds(kerwa_AOI)
               

# Total Number of Images in Image Collection

In [12]:
print('Total number of images :', sentinel_AOI.size().getInfo())

Total number of images : 167


## **NDCI Calculation**

## Iterating over all the images

In [13]:
def addNDCI(image):
  ndci = image.normalizedDifference(['B5', 'B4']).rename('NDCI')
  return image.addBands(ndci)
with_ndci = sentinel_AOI.map(addNDCI)  

## Calculating the Average NDCI on every image in the image collection

In [14]:
def meanNDCI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDCI']),
                           geometry = kerwa_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDCI').getInfo()

In [15]:
Images_ndci = with_ndci.select('NDCI').toList(with_ndci.size())
ndci_coll = []
for i in range(Images_ndci.length().getInfo()):
  image = ee.Image(Images_ndci.get(i-1))
  temp_ndci = meanNDCI(image)
  ndci_coll.append(temp_ndci)

## Getting Dates from the Image Collection

In [16]:
dates = np.array(with_ndci.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

## Creating the Dataframe

In [17]:
df = pd.DataFrame(ndci_coll, index = day, columns = ['Chlorophyll'])
df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
df.sort_index(ascending = True, inplace = True)

In [18]:
df

,Chlorophyll
2018-12-18,0.039289
2018-12-23,0.028044
2018-12-28,0.025008
2019-01-02,0.036703
2019-01-07,0.062004
...,...
2022-12-02,0.022974
2022-12-07,0.024822
2022-12-17,0.029978
2022-12-22,0.024113


In [19]:
df.to_csv('Chlorophyll.csv')

## **NDTI Calculation**

## Iterating over all the images

In [20]:
def addNDTI(image):
  ndti = image.normalizedDifference(['B4', 'B3']).rename('NDTI')
  return image.addBands(ndti)
with_ndti = sentinel_AOI.map(addNDTI)

## Calculating the Average NDTI on every image in the image collection

In [21]:
def meanNDTI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDTI']),
                           geometry = kerwa_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDTI').getInfo()

In [22]:
Images_ndti = with_ndti.select('NDTI').toList(with_ndti.size())
ndti_coll = []
for i in range(Images_ndti.length().getInfo()):
  image = ee.Image(Images_ndti.get(i-1))
  temp_ndti = meanNDTI(image)
  ndti_coll.append(temp_ndti)

## Getting Dates from the Image Collection

In [23]:
dates = np.array(with_ndti.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

## Creating the Dataframe

In [24]:
df1 = pd.DataFrame(ndti_coll, index = day, columns = ['Turbidity'])
df1.index = pd.to_datetime(df1.index, format="%Y/%m/%d")
df1.sort_index(ascending = True, inplace = True)

In [25]:
df1

,Turbidity
2018-12-18,-0.033586
2018-12-23,-0.086675
2018-12-28,-0.081043
2019-01-02,-0.063228
2019-01-07,-0.083032
...,...
2022-12-02,-0.051859
2022-12-07,-0.048762
2022-12-17,-0.035299
2022-12-22,-0.052715


In [26]:
df1.to_csv('Turbidity.csv')

## **NDSI Calculation**

## Iterating over all the images

In [27]:
def addNDSI(image):
  ndsi = image.normalizedDifference(['B11', 'B12']).rename('NDSI')
  return image.addBands(ndsi)
with_ndsi = sentinel_AOI.map(addNDSI)

## Calculating the Average NDSI on every image in the image collection

In [28]:
def meanNDSI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDSI']),
                           geometry = kerwa_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDSI').getInfo()

In [29]:
Images_ndsi = with_ndsi.select('NDSI').toList(with_ndsi.size())
ndsi_coll = []
for i in range(Images_ndsi.length().getInfo()):
  image = ee.Image(Images_ndsi.get(i-1))
  temp_ndsi = meanNDSI(image)
  ndsi_coll.append(temp_ndsi)

## Getting Dates from the Image Collection

In [30]:
dates = np.array(with_ndsi.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

## Creating a Dataframe

In [31]:
df2 = pd.DataFrame(ndsi_coll, index = day, columns = ['Salinty'])
df2.index = pd.to_datetime(df2.index, format="%Y/%m/%d")
df2.sort_index(ascending = True, inplace = True)

In [32]:
df2

,Salinty
2018-12-18,0.042684
2018-12-23,0.135167
2018-12-28,0.075882
2019-01-02,0.114781
2019-01-07,0.095950
...,...
2022-12-02,0.018572
2022-12-07,0.016829
2022-12-17,0.024815
2022-12-22,0.021098


In [33]:
df2.to_csv('Salinty.csv')

# **Dissloved Oxygen Calculation**

## Iterating over all the images

In [34]:
def addDO(image):
  do = image.normalizedDifference(['B4', 'B5']).rename('DO')
  return image.addBands(do)
with_do = sentinel_AOI.map(addDO)

## Calculating the Average Dissolved Oxygen Calculation on every image in the image collection

In [35]:
def meanDO(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['DO']),
                           geometry = kerwa_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('DO').getInfo()

In [36]:
Images_do = with_do.select('DO').toList(with_do.size())
do_coll = []
for i in range(Images_do.length().getInfo()):
  image = ee.Image(Images_do.get(i-1))
  temp_do = meanDO(image)
  do_coll.append(temp_do)

## Getting Dates from the Image Collection

In [37]:
dates = np.array(with_do.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

## Creating a Dataframe

In [38]:
df4 = pd.DataFrame(do_coll, index = day, columns = ['Dissolved Oxygen'])
df4.index = pd.to_datetime(df4.index, format="%Y/%m/%d")
df4.sort_index(ascending = True, inplace = True)

In [39]:
df4

,Dissolved Oxygen
2018-12-18,-0.039604
2018-12-23,-0.029041
2018-12-28,-0.026281
2019-01-02,-0.037898
2019-01-07,-0.063354
...,...
2022-12-02,-0.023392
2022-12-07,-0.025251
2022-12-17,-0.030414
2022-12-22,-0.024570


In [40]:
df4.to_csv('Dissolved Oxygen.csv')

# **pH**

## Iterating over all the images

In [41]:
def addpH(image):
  ph = ee.Image(8.339).subtract(ee.Image(0.827).multiply(image.select('B1').divide(image.select('B8')))).rename('PH')
  return image.addBands(ph)
with_pH = sentinel_AOI.map(addpH)

## Calculating the Average pH on every image in the image collection

In [42]:
def meanpH(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['PH']),
                           geometry = kerwa_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('PH').getInfo()

In [43]:
Images_ph = with_pH.select('PH').toList(with_pH.size())
ph_coll= []
for i in range(Images_ph.length().getInfo()):
  image = ee.Image(Images_ph.get(i-1))
  temp_ph = meanpH(image)
  ph_coll.append(temp_ph)

## Getting Dates from the Image Collection

In [44]:
dates = np.array(with_pH.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

## Creating a Dataframe

In [45]:
df6 = pd.DataFrame(ph_coll, index = day, columns = ['pH'])
df6.index = pd.to_datetime(df6.index, format="%Y/%m/%d")
df6.sort_index(ascending = True, inplace = True)

In [46]:
df6

,pH
2018-12-18,7.533663
2018-12-23,7.054899
2018-12-28,7.232856
2019-01-02,7.386054
2019-01-07,7.533278
...,...
2022-12-02,7.455745
2022-12-07,7.500948
2022-12-17,7.567418
2022-12-22,7.492178


In [47]:
df6.to_csv('pH.csv')

# **Temperature**


## Selecting the Satellite



In [48]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").\
              filterDate("2016-01-01","2022-01-01")

landsat_AOI = landsat.filterBounds(kerwa_AOI)

In [49]:
print('Total number of images :', landsat_AOI.size().getInfo())

Total number of images : 230


## Iterating over all the images

In [50]:
def addtemp(image):
  temp = image.select('ST_B.*').multiply(0.00341802).add(149.0).subtract(273.15).rename('temperature')
  return image.addBands(temp)
with_temp = landsat_AOI.map(addtemp)


## Calculating the Average Temperature on every image in the image collection

In [51]:
def meantemp(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['temperature']),
                           geometry = kerwa_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('temperature').getInfo()

In [52]:
Images_temp = with_temp.select('temperature').toList(with_temp.size())
temp_coll = []
for i in range(Images_temp.length().getInfo()):
  image = ee.Image(Images_temp.get(i-1))
  tempe_temp = meantemp(image)
  temp_coll.append(tempe_temp)

# Getting Dates from the Image Collection

In [53]:
dates = np.array(with_temp.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

## Creating a Dataframe

In [54]:
df5 = pd.DataFrame(temp_coll, index = day, columns = ['Temperature'])
df5.index = pd.to_datetime(df5.index, format="%Y/%m/%d")
df5.sort_index(ascending = True, inplace = True)

In [55]:
df5

,Temperature
2016-01-07,13.566692
2016-01-14,21.774917
2016-01-23,24.049661
2016-01-30,24.262205
2016-02-08,22.965972
...,...
2021-11-27,23.658826
2021-12-06,25.975839
2021-12-13,24.656689
2021-12-22,24.440486


In [56]:
df5.to_csv('Temperature.csv')

# **Dissolved Oxygen Matter**

## Selecting the Satellite

**Data collection through this satellite takes too long, try to give dates between short rangs as monthly or quartly**

In [57]:
sentinel3 = ee.ImageCollection("COPERNICUS/S3/OLCI").\
              filterDate("2018-01-01","2023-01-01")
sentinel3_AOI = sentinel3.filterBounds(kerwa_AOI)

In [58]:
print('Total number of images :', sentinel3_AOI.size().getInfo())

Total number of images : 4504


## Iterating over all the images

In [59]:
def addDM(image):
  rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
  dm = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
  return image.addBands(dm)
with_dm = sentinel3_AOI.map(addDM)

## Calculating the Average Dissolved Oxygen matter on every image in the image collection

In [60]:
def meanDM(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['dom']),
                           geometry = kerwa_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('dom').getInfo()

In [ ]:
Images_dm = with_dm.select('dom').toList(with_dm.size())
dm_coll= []
for i in range(Images_dm.length().getInfo()):
  image = ee.Image(Images_dm.get(i-1))
  temp_dm = meanDM(image)
  dm_coll.append(temp_dm)

## Getting Dates from the Image Collection

In [ ]:
dates = np.array(with_dm.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

## Creating a Dataframe

In [ ]:
df6 = pd.DataFrame(dm_coll, index = day, columns = ['Dissolved Oxygen Matter'])
df6.index = pd.to_datetime(df6.index, format="%Y/%m/%d")
df6.sort_index(ascending = True, inplace = True)

In [ ]:
df6

,Dissolved Oxygen Matter
2020-12-02,0.466159
2020-12-03,0.466518
2020-12-04,0.560359
2020-12-05,0.488783
2020-12-06,0.394080
...,...
2021-11-27,0.453826
2021-11-28,0.454178
2021-11-29,0.448767
2021-11-29,0.381007


In [ ]:
df6.to_csv('Dissolved Oxygen Matter.csv')

# **Suspended Matter**

Selecting the Satellite

**Data collection through this satellite takes too long, try to give dates between short rangs as monthly or quartly** 

In [ ]:
sentinel3 = ee.ImageCollection("COPERNICUS/S3/OLCI").\
              filterDate("2018-01-01","2023-01-01")
sentinel3_AOI = sentinel3.filterBounds(kerwa_AOI)

In [ ]:
print('Total number of images :', sentinel3_AOI.size().getInfo())

Total number of images : 407


## Iterating over all the images

In [ ]:
def addSM(image):
  rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
  suspended_matter = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')
  return image.addBands(suspended_matter)
with_sm = sentinel3_AOI.map(addSM)

## Calculating the Average Suspended Matter on every image in the image collection

In [ ]:
def meanSM(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['suspended_matter']),
                           geometry = kerwa_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('suspended_matter').getInfo()

In [ ]:
Images_sm = with_sm.select('suspended_matter').toList(with_sm.size())
sm_coll= []
for i in range(Images_sm.length().getInfo()):
  image = ee.Image(Images_sm.get(i-1))
  temp_sm = meanSM(image)
  sm_coll.append(temp_sm)

## Getting Dates from the Image Collection

In [ ]:
dates = np.array(with_sm.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

## Creating a Dataframe

In [ ]:
df7 = pd.DataFrame(sm_coll, index = day, columns = ['Suspended Matter'])
df7.index = pd.to_datetime(df7.index, format="%Y/%m/%d")
df7.sort_index(ascending = True, inplace = True)

In [ ]:
df7

,Suspended Matter
2018-04-02,NaN
2018-04-03,0.601059
2018-04-03,NaN
2018-04-03,NaN
2018-04-03,NaN
...,...
2018-04-30,NaN
2018-04-30,NaN
2018-04-30,NaN
2018-04-30,NaN


In [ ]:
df7.to_csv('Suspended Matter.csv')